# Fake News Detection Model


People check out this paper: https://sites.cs.ucsb.edu/~william/papers/acl2017.pdf


### Data properties

    Column 1: the ID of the statement ([ID].json).
    Column 2: the label.
    Column 3: the statement.
    Column 4: the subject(s).
    Column 5: the speaker.
    Column 6: the speaker's job title.
    Column 7: the state info.
    Column 8: the party affiliation.
    Column 9-13: the total credit history count, including the current statement.
    9: barely true counts.
    10: false counts.
    11: half true counts.
    12: mostly true counts.
    13: pants on fire counts.
    Column 14: the context (venue / location of the speech or statement)

**Steps Involved:**

- **Preprocessing**
    - Cleaning
    - Steamming
    - Contraction removal
    - Special character removal
- **EDA (Easiest Data Augmentation)** 
    - Word2Vec embedding enrichment
    - Misspelling removal
    - Feature creation

- **Text Representation**
    - Tokenization
    - Text to sequence
    - Padding sequence

- **Modelling** 
    -  Convention models
    - CNN, Bi-LSTM
    
- **Deployment** 

**_`Text Representation`_**

    - Bags of Words
    - TFIDF (Term Frequency Inverse Document Frequency)
    - Hashing Vectorization
    - Word2Vec


There are two ways to approach the problem on modelling front.

- **Convention Methods (Convention Machine Learning approach)**
    - Logistic Regression
    - Support Vector Machine
    - Navie Bayes Classifier
        
- **Deep Learning Methods**
    - CNN
    - Bi-LSTM
    - Combination of above two ^

## Import dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Text Preprocessing needes
# Bag of words 
from sklearn.feature_extraction.text import CountVectorizer
# TFIDF 
from sklearn.feature_extraction.text import TfidfVectorizer
# Hashing 
from sklearn.feature_extraction.text import HashingVectorizer

# Word2Vec
# Python program to generate word vectors using Word2Vec 
# importing all necessary modules 
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings
warnings.filterwarnings(action = 'ignore') 
import gensim 
from gensim.models import Word2Vec 

from sklearn.preprocessing import LabelEncoder



# !python3 -m pip install wordcloud
#from wordcloud import WordCloud, STOPWORDS


# Models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# Peformance 
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,recall_score,precision_score,precision_recall_curve,f1_score,confusion_matrix
from sklearn.metrics import roc_auc_score


## Utils 

In [3]:
def display_model_metrics(y_pred,y_test):
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('F1 score:', f1_score(y_test, y_pred,average='weighted'))
    print('Recall:', recall_score(y_test, y_pred, average='weighted'))
    print('Precision:', precision_score(y_test, y_pred,average='weighted'))
    print('\n clasification report:\n', classification_report(y_test, y_pred))
    print('\n confussion matrix:\n',confusion_matrix(y_test, y_pred))

In [4]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### Data Load

In [5]:
columns = ['id','class','statement','subjects','speaker','speaker_title','state','party_affiliation',
           'barely_true_count','false_counts','half_true_counts','mostly_true_counts','pants_on_fire','context' ]

In [6]:
train_raw=pd.read_csv('data/liar_dataset/train.tsv',delimiter='\t',names=columns)
validation_raw=pd.read_csv('data/liar_dataset/valid.tsv',delimiter='\t',names=columns)
test_raw=pd.read_csv('data/liar_dataset/test.tsv',delimiter='\t',names=columns)

In [7]:
train_raw.head(5)

,id,class,statement,subjects,speaker,speaker_title,state,party_affiliation,barely_true_count,false_counts,half_true_counts,mostly_true_counts,pants_on_fire,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [8]:
train_raw[train_raw['speaker']=='mitt-romney'].ix[:,'barely_true_count':'pants_on_fire'].head(1)
# For example, Mitt Romney has a credit
# history vector h = {19, 32, 34, 58, 33}, which corresponds to his counts of “pants on fire”, “false”,
# “barely true”, “half true”, “mostly true” for historical statements

,barely_true_count,false_counts,half_true_counts,mostly_true_counts,pants_on_fire
12,34.0,32.0,58.0,33.0,19.0


In [9]:
train_raw['speaker'].value_counts()

barack-obama                          488
donald-trump                          273
hillary-clinton                       239
mitt-romney                           176
scott-walker                          149
                                     ... 
curt-schilling                          1
jason-nassour                           1
drug-policy-alliance                    1
trusted-leadership-pac                  1
national-association-manufacturers      1
Name: speaker, Length: 2910, dtype: int64

In [10]:
train_raw['class'].value_counts()

half-true      2114
false          1995
mostly-true    1962
true           1676
barely-true    1654
pants-fire      839
Name: class, dtype: int64

In [11]:
train_raw['state'].value_counts()

Texas            1009
Florida           997
Wisconsin         713
New York          657
Illinois          556
                 ... 
Tennesse            1
Montana             1
ohio                1
Wisconsin           1
Rhode Island        1
Name: state, Length: 84, dtype: int64

In [12]:
train_raw['party_affiliation'].value_counts()

republican                      4497
democrat                        3336
none                            1744
organization                     219
independent                      147
newsmaker                         56
libertarian                       40
activist                          39
journalist                        38
columnist                         35
talk-show-host                    26
state-official                    20
labor-leader                      11
tea-party-member                  10
business-leader                    9
green                              3
education-official                 2
Moderate                           1
liberal-party-canada               1
government-body                    1
constitution-party                 1
ocean-state-tea-party-action       1
democratic-farmer-labor            1
Name: party_affiliation, dtype: int64

In [13]:
train_labels=train_raw[['class']]

In [14]:
train_labels.head()

,class
0,false
1,half-true
2,mostly-true
3,false
4,half-true


In [15]:
target_classes = train_labels['class'].unique()

In [16]:
target_classes

array(['false', 'half-true', 'mostly-true', 'true', 'barely-true',
       'pants-fire'], dtype=object)

In [18]:
train =train_raw

In [20]:
train.head(2)

,id,class,statement,subjects,speaker,speaker_title,state,party_affiliation,barely_true_count,false_counts,half_true_counts,mostly_true_counts,pants_on_fire,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.


In [21]:
## Remvoing ID from data because we are not able get text data correspding to ID json
## Remvoving target class 
train.drop(['id','class'], axis=1,inplace=True)

In [22]:
train.head()

,statement,subjects,speaker,speaker_title,state,party_affiliation,barely_true_count,false_counts,half_true_counts,mostly_true_counts,pants_on_fire,context
0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


### Word embedding word2vec 

https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/

https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-11-cnn-word2vec-41f5e28eda74


Note: Only considering statement column as actual text which is available in ID filed is not accessible via API.


In [23]:
y_train = train_labels
df2= test_raw[['statement']]
df2_labels= test_raw[['class']]
y_test = df2_labels

## Convetionl Methods to set up a base line

### Text Representation

- Bag of words (CountVectorizer)
- TFIDF 
- Hashing
- Word2Vec

### Models to Try

- Naive Bayes Classifer
- Logistic Regression
- Support Vector Machines


In [24]:
train_df= train
test_df= df2

### Bag of words (CountVectorizer)

In [25]:
cnt_vectorizer = CountVectorizer(dtype=np.float32,
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),min_df=3)

In [26]:
cnt_vectorizer.fit(list(train_df.statement.values) + list(test_df.statement.values))
xtrain_cntv =  cnt_vectorizer.transform(train_df.statement.values) 
xtest_cntv = cnt_vectorizer.transform(test_df.statement.values)

In [27]:
len(cnt_vectorizer.get_feature_names())

22869

In [28]:
xtrain_cntv.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [29]:
xtrain_cntv.shape

(10240, 22869)

### LogisticRegression

In [30]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_cntv,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
y_pred= clf.predict(xtest_cntv)

In [32]:
display_model_metrics(y_pred,y_test)

Accuracy: 0.244672454617206
F1 score: 0.24170286932296786
Recall: 0.244672454617206
Precision: 0.2421979288134382

 clasification report:
               precision    recall  f1-score   support

 barely-true       0.23      0.19      0.21       212
       false       0.29      0.35      0.31       249
   half-true       0.22      0.21      0.22       265
 mostly-true       0.23      0.24      0.23       241
  pants-fire       0.24      0.14      0.18        92
        true       0.25      0.27      0.26       208

    accuracy                           0.24      1267
   macro avg       0.24      0.23      0.23      1267
weighted avg       0.24      0.24      0.24      1267


 confussion matrix:
 [[41 51 49 32 11 28]
 [37 86 36 43  9 38]
 [36 60 56 66  9 38]
 [32 37 52 58  7 55]
 [13 31 19  9 13  7]
 [23 35 39 49  6 56]]


### Multinomial Naive Bayes 

In [33]:
modelNB = MultinomialNB(alpha=0.1)
modelNB.fit(xtrain_cntv, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [34]:
y_pred = modelNB.predict(xtest_cntv)

In [35]:
display_model_metrics(y_pred,y_test)

Accuracy: 0.2541436464088398
F1 score: 0.2546755622193559
Recall: 0.2541436464088398
Precision: 0.25654155347626184

 clasification report:
               precision    recall  f1-score   support

 barely-true       0.23      0.22      0.23       212
       false       0.29      0.27      0.28       249
   half-true       0.26      0.24      0.25       265
 mostly-true       0.26      0.27      0.26       241
  pants-fire       0.18      0.25      0.21        92
        true       0.26      0.27      0.27       208

    accuracy                           0.25      1267
   macro avg       0.25      0.25      0.25      1267
weighted avg       0.26      0.25      0.25      1267


 confussion matrix:
 [[47 42 41 26 31 25]
 [35 66 43 36 29 40]
 [42 42 64 65 15 37]
 [31 18 52 65 15 60]
 [22 21 12 12 23  2]
 [23 36 34 46 12 57]]


### Support Vector Machine 

In [36]:
svc = LinearSVC()
svc.fit(xtrain_cntv, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [37]:
y_pred = svc.predict(xtest_cntv)

In [38]:
display_model_metrics(y_pred,y_test)

Accuracy: 0.2280978689818469
F1 score: 0.2272794496328766
Recall: 0.2280978689818469
Precision: 0.22728152796159046

 clasification report:
               precision    recall  f1-score   support

 barely-true       0.21      0.21      0.21       212
       false       0.28      0.30      0.29       249
   half-true       0.22      0.18      0.20       265
 mostly-true       0.21      0.23      0.22       241
  pants-fire       0.20      0.18      0.19        92
        true       0.23      0.23      0.23       208

    accuracy                           0.23      1267
   macro avg       0.22      0.22      0.22      1267
weighted avg       0.23      0.23      0.23      1267


 confussion matrix:
 [[45 46 34 39 17 31]
 [42 75 33 46 17 36]
 [39 57 49 65 15 40]
 [42 28 59 55  8 49]
 [18 23 17 10 17  7]
 [28 36 35 49 12 48]]


##### Note : Looks like Navie Bayes Classifer has performed better than SVM and LG

### TFIDF (Term Frequency Inverse Document Frequency)

In [39]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(dtype=np.float32, min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(train_df.statement.values) + list(test_df.statement.values))
xtrain_tfv =  tfv.transform(train_df.statement.values) 
xvalid_tfv = tfv.transform(test_df.statement.values)

In [40]:
len(tfv.get_feature_names())

10546

In [41]:
xtrain_tfv.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [42]:
xtrain_tfv.shape

(10240, 10546)

###### Note : The number for features are reduce by half if we use TFIDF

In [43]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_cntv,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
y_pred= clf.predict(xtest_cntv)

In [45]:
display_model_metrics(y_pred,y_test)

Accuracy: 0.244672454617206
F1 score: 0.24170286932296786
Recall: 0.244672454617206
Precision: 0.2421979288134382

 clasification report:
               precision    recall  f1-score   support

 barely-true       0.23      0.19      0.21       212
       false       0.29      0.35      0.31       249
   half-true       0.22      0.21      0.22       265
 mostly-true       0.23      0.24      0.23       241
  pants-fire       0.24      0.14      0.18        92
        true       0.25      0.27      0.26       208

    accuracy                           0.24      1267
   macro avg       0.24      0.23      0.23      1267
weighted avg       0.24      0.24      0.24      1267


 confussion matrix:
 [[41 51 49 32 11 28]
 [37 86 36 43  9 38]
 [36 60 56 66  9 38]
 [32 37 52 58  7 55]
 [13 31 19  9 13  7]
 [23 35 39 49  6 56]]


###### Note: Same way we can try other models

### Hashing Features

### Word2vec Features

Here comes the most imporant and widely used technique for text embedding. Word2vec converts word to a vector of dimension 'd'.

- Word to vector of dimension 'd'
- Can be used for both Coventation and Deep Learning modesl 

In [47]:
train.head(2)

,statement,subjects,speaker,speaker_title,state,party_affiliation,barely_true_count,false_counts,half_true_counts,mostly_true_counts,pants_on_fire,context
0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.


In [48]:
sent = [row.split(' ') for row in train['statement']]

In [49]:
model_ug_sg = Word2Vec(sent, min_count=1,size= 50,workers=3, window =3, sg = 1)
model_ug_cbow = Word2Vec(sent, min_count=1,size= 50,workers=3, window =3, sg = 0)

In [52]:
model_ug_sg['Department']

array([ 0.19703138, -0.061311  , -0.40880716, -0.49867463,  0.03541506,
       -0.35453823,  0.30740044, -0.06979024, -0.3306938 ,  0.29036957,
       -0.39855817, -0.350866  , -0.07147917,  0.3059135 ,  0.02957194,
        0.04645884,  0.3203365 ,  1.3326272 ,  0.00597334,  0.18950194,
       -0.16745216, -0.17665003, -0.56367654, -0.30047864, -0.3120408 ,
       -0.23330745,  0.2790714 , -0.12381896, -0.36315352, -0.5928923 ,
       -0.3191158 , -0.04845972,  0.44939926,  0.3536869 , -0.41129047,
       -0.16447859, -0.10676751,  0.23111168,  0.01500688,  0.04714711,
       -0.01127017,  0.80032605, -0.49741605,  0.22328955,  0.3381129 ,
       -0.11633526, -0.19679421,  0.17576802,  0.29594406,  0.44566664],
      dtype=float32)

In [54]:
model_ug_sg.corpus_count

10240

In [55]:
model_ug_sg.corpus_total_words

184014

In [56]:
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])

In [57]:
embeddings_index['statement'].shape

(100,)

In [58]:
train.columns

Index(['statement', 'subjects', 'speaker', 'speaker_title', 'state',
       'party_affiliation', 'barely_true_count', 'false_counts',
       'half_true_counts', 'mostly_true_counts', 'pants_on_fire', 'context'],
      dtype='object')

In [59]:
train[:1]

,statement,subjects,speaker,speaker_title,state,party_affiliation,barely_true_count,false_counts,half_true_counts,mostly_true_counts,pants_on_fire,context
0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer


### Tokenizer Keras 

In [60]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(train.statement)
sequences = tokenizer.texts_to_sequences(train.statement)

Using TensorFlow backend.


In [76]:
sequences[:1]

[[7, 1, 6968, 1141, 520, 621, 385, 444, 5119, 585, 11, 1601]]

Note: each sequence is having diffirent length, lets pad it

In [77]:
length = []
for x in train.statement:
    length.append(len(x.split()))
max(length)

467

In [78]:
x_train_seq = pad_sequences(sequences, maxlen=467)
x_train_seq[:5]

array([[   0,    0,    0, ...,  585,   11, 1601],
       [   0,    0,    0, ...,  560, 1365,  177],
       [   0,    0,    0, ..., 3547,   11,  416],
       [   0,    0,    0, ...,  467,  417, 4148],
       [   0,    0,    0, ...,    3,  174,  505]], dtype=int32)

In [64]:
print('Shape of data tensor:', x_train_seq.shape)

Shape of data tensor: (10240, 467)


In [67]:
test = test_df

In [84]:
sequences_val = tokenizer.texts_to_sequences(validation_raw.statement)
x_val_seq = pad_sequences(sequences_val, maxlen=467)

In [86]:
x_val_seq.shape

(1284, 467)

In [87]:
num_words = 100000
embedding_matrix = np.zeros((num_words, 100))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [88]:
np.array_equal(embedding_matrix[1] ,embeddings_index.get('the'))

True

In [89]:
np.array_equal(embedding_matrix[6968] ,embeddings_index.get('Annies'))

False

In [90]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.65764731, -0.02344695, -1.40152967, ...,  0.18532071,
         0.29149911,  0.12424401],
       [-1.08712626, -0.01805102, -1.25420547, ...,  0.291509  ,
         0.31132948,  0.6916548 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [111]:
# https://www.kaggle.com/yekenot/2dcnn-textclassifier
def model_cnn(embedding_matrix):
    filter_sizes = [1,2,3,5]
    num_filters = 36

    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Reshape((maxlen, embed_size, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size),
                                     kernel_initializer='he_normal', activation='relu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [112]:
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf

## TODO - Next steps is to try CNN, Bi-LSTM, etc